# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [26]:
# Import dependencies
from pymongo import MongoClient, DESCENDING
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_databases())

In [4]:
# assign the uk_food database to a variable name
uk_food = mongo["uk_food"]

In [5]:
# review the collections in our new database
print(uk_food.list_collection_names())

['establishments']


In [15]:
# review the collections in our new database
establishments = uk_food["establishments"]



In [16]:
# review a document in the establishments collection
establishments.find()[0]

{'_id': ObjectId('6450198c0b1568a13b189c50'),
 'FHRSID': 254719,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT15 6DY',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2022-03-24T00:00:00',
 'LocalAuthorityCode': '182',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'scores': {'Hygiene': 0, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.387974', 'latitude': '51.152225'},
 'RightToReply': '',
 'Distance': 4587.347174863443,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  'itemCount': 0,
  'returncode

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [17]:
# Create a dictionary for the new restaurant data
newRestaurant = {"name": "Penang Flavours"}

In [18]:
# Insert the new restaurant into the collection
establishments.insert_one(newRestaurant )

In [34]:
# Check that the new restaurant was inserted
new_place = establishments.find().sort('_id', DESCENDING).limit(1)[0]
print(new_place)

{'_id': ObjectId('64501c50207c64e4f16ee269'), 'name': 'Penang Flavours'}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [35]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
cafes_type_w_id = establishments.find_one(query,{"BusinessTypeID":1,  "BusinessType":1})

In [40]:
cafes_type_w_id[0]

cafe_business_id = cafes_type_w_id[0]["BusinessTypeID"]

3. Update the new restaurant with the `BusinessTypeID` you found.

In [41]:
# Update the new restaurant with the correct BusinessTypeID
new_place["BusinessTypeID"] = cafe_business_id

In [42]:
# Confirm that the new restaurant was updated
new_place

{'_id': ObjectId('64501c50207c64e4f16ee269'),
 'name': 'Penang Flavours',
 'BusinessTypeID': 1}

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [45]:
# Find how many documents have LocalAuthorityName as "Dover"
leader_named_dover_count = establishments.count_documents({"LocalAuthorityName" :"Dover"})
leader_named_dover_count

994

In [49]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({"LocalAuthorityName" :"Dover"})

In [50]:
# Check if any remaining documents include Dover
new_leader_named_dover_count = establishments.count_documents({"LocalAuthorityName" :"Dover"})
new_leader_named_dover_count

0

In [52]:
# Check that other documents remain with 'find_one'
establishments.find_one()

{'_id': ObjectId('6450198c0b1568a13b189f36'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.195625', 'latitude': '51.083812'},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01

5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [56]:
# Change the data type from String to Decimal for longitude
establishments.update_many({"longitude":{"$type" : "string"}}, { "$set": { "longitude": { "$toDeciaml": "$longitude" } } })

In [57]:
# Change the data type from String to Decimal for latitude
establishments.update_many({"latitude":{"$type" : "string"}}, { "$set": { "latitude": { "$toInt": "$latitude" } } })


In [69]:
# Check that the coordinates are now numbers
print(establishments.find_one())


{'_id': ObjectId('6450198c0b1568a13b189f36'), 'FHRSID': 1043695, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000073616', 'BusinessName': 'The Pavilion', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'East Cliff Pavilion', 'AddressLine2': 'Wear Bay Road', 'AddressLine3': 'Folkestone', 'AddressLine4': 'Kent', 'PostCode': 'CT19 6BL', 'Phone': '', 'RatingValue': '5', 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2018-04-04T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.195625', 'latitude': '51.083812'}, 'RightToReply': '', 'Distance': 4591.765489457773, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 